In [14]:
import requests
from bs4 import BeautifulSoup

In [61]:
def scrape_the_list(url):
    try:
        response = requests.get(url)

        if response.status_code != 200:
            print(f"Failed to get the webpage: {response.status_code}")
            return []
        
        soup = BeautifulSoup(response.text, 'html.parser')

        watches_list = soup.find_all('section', class_ = "CategorySearchCard__CategorySearchCardGrid-sc-1o7izf2-1 dliokm")

        watches_dict = dict()

        for index, watch in enumerate(watches_list, 1):
            
            item_name = soup.find('span', class_ = 'hui-text-body-primary text-text-primary').text

            item_price_str = soup.find('span', class_ = 'FormattedCurrency__StyledFormattedCurrency-sc-1ugrxi1-0 cqnbDD').text
            
            watches_dict[index] = {}
            watches_dict[index]["Name"] = item_name
            watches_dict[index]["Price"] = item_price_str

        return watches_dict

    except Exception as e:
        print(f"Error has occurred: {e}")
        return []    

def get_watches(url):
    # url: string of https URL from liveauctioneers.com

    results = scrape_the_list(url)

    return results

In [62]:
url = "https://www.liveauctioneers.com/c/watches/97/"

watches = get_watches(url)

In [63]:
print(watches)

{1: {'Name': 'Wakmann Skin Diver 356945', 'Price': '$550'}, 2: {'Name': 'Wakmann Skin Diver 356945', 'Price': '$550'}, 3: {'Name': 'Wakmann Skin Diver 356945', 'Price': '$550'}, 4: {'Name': 'Wakmann Skin Diver 356945', 'Price': '$550'}, 5: {'Name': 'Wakmann Skin Diver 356945', 'Price': '$550'}, 6: {'Name': 'Wakmann Skin Diver 356945', 'Price': '$550'}, 7: {'Name': 'Wakmann Skin Diver 356945', 'Price': '$550'}, 8: {'Name': 'Wakmann Skin Diver 356945', 'Price': '$550'}, 9: {'Name': 'Wakmann Skin Diver 356945', 'Price': '$550'}, 10: {'Name': 'Wakmann Skin Diver 356945', 'Price': '$550'}, 11: {'Name': 'Wakmann Skin Diver 356945', 'Price': '$550'}, 12: {'Name': 'Wakmann Skin Diver 356945', 'Price': '$550'}, 13: {'Name': 'Wakmann Skin Diver 356945', 'Price': '$550'}, 14: {'Name': 'Wakmann Skin Diver 356945', 'Price': '$550'}, 15: {'Name': 'Wakmann Skin Diver 356945', 'Price': '$550'}, 16: {'Name': 'Wakmann Skin Diver 356945', 'Price': '$550'}, 17: {'Name': 'Wakmann Skin Diver 356945', 'Price